In [1]:
from IntroUtils import *
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [2]:
df = pd.read_csv('../../Utils/Datasets/Amazon-cellphone-reviews/Amazon_Unlocked_Mobile.csv')
#df = df.sample(frac=0.1, random_state=10)
df.dropna(inplace=True)
df = df[df['Rating'] != 3]
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df = df[['Reviews','Positively Rated']]
df['Reviews'] = df['Reviews'].str.lower()
#df = df.iloc[:10,:]

In [4]:
dataset = TextClassificationDataset.from_dataframe(df)

In [20]:
import itertools

class Vocabulary(object):
    pass


class TextClassificationDataset(Dataset):
    
    def __init__(self, corpus_examples_idx, vocab_examples, corpus_labels_idx, vocab_labels, encoding):
        self.corpus_examples_idx = corpus_examples_idx
        self.vocab_examples = vocab_examples
        self.corpus_labels_idx = corpus_labels_idx
        self.vocab_labels = vocab_labels
        
        if encoding == 'index':
            self.set_getitem(lambda self, idx: self.corpus_examples_idx[idx])
        elif encoding == 'one-hot':
            pass
        elif encoding == 'tf-idf':
            pass
        elif encoding == 'ppmi':
            pass
        else:
            raise ValueError('Encodificación no válida')
        
    @classmethod
    def set_getitem(cls, getitem_fn):
        cls.__getitem__ = getitem_fn
    
    @staticmethod
    def get_vocabulary_from_list_of_list_of_tokens(corpus,words=None):
        if words is None:
            words = sorted(list(set(itertools.chain.from_iterable(corpus))))
        else:
            words = sorted(words)
        freqs_idx_dict = {tk: [0,idx] for idx, tk in enumerate(words)}
        for tk in itertools.chain.from_iterable(corpus):
            try:
                freqs_idx_dict[tk][0] += 1
            except KeyError:
                continue
        return freqs_idx_dict
    
    @classmethod
    def from_dataframe(cls,df,tokenizer=None,vocabulary=None,encoding='index'):
        if tokenizer is None:
            tokenizer = lambda x: x.split(' ')
        corpus_examples = df.iloc[:,0].apply(tokenizer).tolist()
        vocab_examples = cls.get_vocabulary_from_list_of_list_of_tokens(corpus_examples,words=vocabulary)
        corpus_examples_idx = [[vocab_examples.get(tk,-1)[1] for tk in doc] for doc in corpus_examples]
        
        corpus_labels = df.iloc[:,1].tolist()
        vocab_labels = sorted(list(set(corpus_labels)))
        corpus_labels_idx = [vocab_labels.index(tk) for tk in corpus_labels]
        
        return cls(corpus_examples_idx, vocab_examples, corpus_labels_idx, vocab_labels, encoding)
    
        
dataset = TextClassificationDataset.from_dataframe(df)

In [7]:
getsize(dataset) / 1000 / 1000

165.234884

In [8]:
df.iloc[0,0]

"i feel so lucky to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. my son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! thank you seller, we really appreciate it & your honesty re: said used phone.i recommend this seller very highly & would but from them again!!"

In [9]:
idx_to_tk = {idx:tk for idx,tk in enumerate(dataset.vocab_examples.keys())}
print([idx_to_tk[idx] for idx in dataset[0]])
    

['i', 'feel', 'so', 'lucky', 'to', 'have', 'found', 'this', 'used', '(phone', 'to', 'us', '&', 'not', 'used', 'hard', 'at', 'all),', 'phone', 'on', 'line', 'from', 'someone', 'who', 'upgraded', 'and', 'sold', 'this', 'one.', 'my', 'son', 'liked', 'his', 'old', 'one', 'that', 'finally', 'fell', 'apart', 'after', '2.5+', 'years', 'and', "didn't", 'want', 'an', 'upgrade!!', 'thank', 'you', 'seller,', 'we', 'really', 'appreciate', 'it', '&', 'your', 'honesty', 're:', 'said', 'used', 'phone.i', 'recommend', 'this', 'seller', 'very', 'highly', '&', 'would', 'but', 'from', 'them', 'again!!']


In [29]:
n = 10000
d = 200
t = 128
device = torch.device('cuda:1')
emb = nn.Embedding(n,d).to(device)
lin = nn.Linear(n,d).to(device)

def f1(in_emb):
    return torch.randint(n,(t,),device=device)

def f2(in_emb):
in_lin = torch.zeros(t,n, device=device)
for i in range(t):
    in_lin[i,in_emb[i]] = 1

%timeit lin(in_lin).sum(dim=1)
%timeit emb(in_emb).sum(dim=1)                     

177 µs ± 8.03 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)
23.5 µs ± 58.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [27]:
df = pd.read_csv('../../Utils/Datasets/Amazon-cellphone-reviews/Amazon_Unlocked_Mobile.csv')
#df = df.sample(frac=0.1, random_state=10)
df.dropna(inplace=True)
df = df[df['Rating'] != 3]
df['Positively Rated'] = np.where(df['Rating'] > 3, 1, 0)
df = df[['Reviews','Positively Rated']]
df['Reviews'] = df['Reviews'].str.lower()
#df = df.iloc[:10,:]

In [ ]:
np.random.seed(2635147587)

data_size = len(df)
train_size, val_size = .85, .05
perm = np.random.permutation(data_size)
train_idx, val_idx, test_idx = perm[:int(train_size*data_size)], perm[int(train_size*data_size):int((train_size+val_size)*data_size)], perm[int((train_size+val_size)*data_size):]
train_data, val_data, test_data = df.iloc[train_idx], df.iloc[val_idx], df.iloc[test_idx]

In [6]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [28]:
tokenizer = lambda x: x.split(' ')

corpus_df = df.iloc[:,0].apply(tokenizer)
corpus_list = df.iloc[:,0].apply(tokenizer).tolist()
tk_to_idx = {tk:idx for idx,tk in enumerate(set([tk for doc in corpus_list for tk in doc]))}
corpus_idx = [tk_to_idx[tk] for doc in corpus_list for tk in doc]
#print(getsize(corpus_df)/1000 / 1000)
print(getsize(corpus_list)/1000/1000)
print((getsize(corpus_idx)+getsize(tk_to_idx))/1000/1000)

747.5185749999999
124.058352


In [26]:
def f(corpus):
    for doc in corpus:
        for tk in corpus:
            str(tk)
tic = time.time()
f(corpus_df)
toc = time.time()
print(toc - tic)
tic = time.time()
f(corpus_list)
toc = time.time()
print(toc - tic)
tic = time.time()
f(corpus_idx)
toc = time.time()
print(toc - tic)

0.0003070831298828125
0.000186920166015625
0.006488800048828125


In [12]:
import time
time.time()

1584053210.5393717

In [18]:
tokenizer = lambda x: x.split(' ')
c = df.iloc[:,0].apply(tokenizer).tolist()
corpus_words = set([tk for doc in c for tk in doc])
freqs_dict = {word: [0,idx] for idx,word in enumerate(corpus_words)}
for tk in itertools.chain.from_iterable(c):
    freqs_dict[tk][0] += 1
corpus = [[freqs_dict[tk][1] for tk in doc] for doc in c]

In [19]:
print(getsize(df.iloc[:,0]) / 1000 )
print(getsize(c) / 1000 )
print((getsize(corpus)+getsize(freqs_dict)) / 1000 )

3.954
15.585
28.488


AttributeError: 'Vocabulary' object has no attribute 'max_idx'

In [2]:
print('Cantidad de muestras de train:',len(train_data))
print('Cantidad de muestras de validación:',len(val_data))
print('Cantidad de muestras de test:',len(test_data))

vocabulary = Vocabulary.from_dataseries(df.iloc[:,0])
train_dataset = TextClassificationDataset.from_dataframe(train_data, tokenizer=None, vocabulary=vocabulary)
val_dataset = TextClassificationDataset.from_dataframe(val_data, tokenizer=None, vocabulary=vocabulary)
test_dataset = TextClassificationDataset.from_dataframe(test_data, tokenizer=None, vocabulary=vocabulary)

Cantidad de muestras de train: 262035
Cantidad de muestras de validación: 15414
Cantidad de muestras de test: 30828


In [6]:
print(len(train_dataset))
print(len(train_dataset.vocab_x))
print(len(train_dataset[0][0]))
print(len(vocabulary))

262035
262035
5313
185508


In [5]:
class LogisticRegressionTrainer(Trainer):
    
    def __init__(self, train_dataset, val_dataset=None, test_dataset=None, batch_size=128, device='cpu'):
        vocab_size = len(train_dataset.vocab_x)
        model = self.Model(vocab_size)
        super().__init__(model,train_dataset,val_dataset, test_dataset,batch_size,device)
    
    class Model(nn.Module):
        
        def __init__(self, vocab_size):
            super().__init__()
            self.linear = nn.Embedding(vocab_size+2,1, padding_idx=-1)
        
        def forward(self,x):
            x = torch.cat((x,torch.ones(x.size(0),1,dtype=torch.long,device=x.device)),dim=1)
            return self.linear(x).squeeze().sum(dim=1)
        
    def Loss(self,scores,targets):
        lf = nn.BCEWithLogitsLoss()
        print(scores)
        print(targets)
        return lf(scores,targets)
    
#     def CheckAccuracy(self):
#         num_correct = 0
#         num_samples = 0
#         self.model.eval()
#         with torch.no_grad():
#             for x, y in self.val_dataloader:
#                 x = x.to(device=self.device)  
#                 y = y.to(device=self.device)

#                 scores = self.model(x)
#                 preds = (torch.sigmoid(scores) > .5).type(torch.long)
#                 num_correct += (preds == y).sum()
#                 num_samples += preds.size(0)
#         self.model.train()
#         return ', Accuracy: {}/{} ({:.1f}%)'.format(num_correct,num_samples,float(num_correct)/float(num_samples)*100)

trainer = LogisticRegressionTrainer(train_dataset, val_dataset, test_dataset, batch_size=128, device='cpu')

In [6]:
algorithm = 'Adam'
epochs = 10
sample_loss_every = 100
lr = 1e-3
trainer.Train(algorithm, epochs, sample_loss_every, lr=lr)

Starting training...
Optimization method: Adam
Learning Rate: 0.001
Number of epochs: 10
Running on device (cpu)



RuntimeError: index out of range: Tried to access index 10326103 out of table with 262035 rows. at /opt/conda/conda-bld/pytorch_1570910687650/work/aten/src/TH/generic/THTensorEvenMoreMath.cpp:418